In [1]:
import asyncio
import difflib
import json
import sys
from functools import wraps

import pandas as pd
import plotly.io
from litellm import acompletion, completion_cost
from pydantic import BaseModel
from tqdm.asyncio import tqdm
from unidecode import unidecode

sys.path.append("../..")

plotly.io.renderers.default = "png"

## Make predictions for one month (07/2024)


In [2]:
# Load data
df = pd.read_parquet("../../data/raw/18_channels_2023_09_to_2024_09.parquet")

# Filter some channels only
df = df[df["channel_name"].isin(["sud-radio", "europe1", "itele"])]
df = df[(df["start"] >= "2024-01-01") & (df["start"] < "2024-08-01")]

# Fixing some syntax issues in the data
# Mediatree issue with "' "
df["text"] = df["text"].str.replace("' ", "'")

df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
ae3110362b2d90e3b39022926052cb8c57d52be76f692dfccb179236a64fdefd,2024-01-01 06:02:00,sur la boisson maintenant que les fêtes sont a...,europe1,True,Information - Magazine,Bonjour,"[""ressources"", ""changement_climatique_constat""...","[{""keyword"": ""transition \u00e9cologique"", ""ti...",1,524
068edefaad4938b6cf7a86bf8e306f6968bbbf3c96e84627b6295900ed9d5615,2024-01-01 06:14:00,première prescription culture de l'année c'est...,europe1,True,Information - Magazine,Bonjour,"[""changement_climatique_constat"", ""biodiversit...","[{""keyword"": ""ademe"", ""timestamp"": 17040861260...",1,497
f647d3fad5acbe70ee52b488bd3e8ec8be1018acc4e7756b48bb29b81b877e50,2024-01-01 06:36:00,hamas hier soir c'était donc un réveillon dans...,europe1,True,Information - Magazine,Bonjour,"[""attenuation_climatique_solutions"", ""biodiver...","[{""keyword"": ""sobri\u00e9t\u00e9"", ""timestamp""...",1,496
d346e52092308430d6ec6faa668722219ce024db394c112e5e79ba9b8352ff89,2024-01-01 06:54:00,d'accélérer cette part du made in france mais ...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""attenuatio...","[{""keyword"": ""emballage"", ""timestamp"": 1704088...",1,488
cb1c730c8eabec84989e98a517db234d22b495d3e0814a9a6286dd61f949d89c,2024-01-01 07:02:00,qui porte bonheur pour moi le quatre août deux...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""feux"", ""timestamp"": 170408896200...",1,528
...,...,...,...,...,...,...,...,...,...,...
32b48450801f677a3834cb84281b9d91d974f8c8d3ca7e84bed103d7218566de,2024-07-31 10:04:00,dans toute la france le bureau de recherches g...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""changement_climatique_constat"", ""changement_...","[{""keyword"": ""agriculteur"", ""timestamp"": 17224...",4,530
2750001e70a1cd4eae8ac6932a495b7ac97feeab1e9d747a53a792b79d4d6a5d,2024-07-31 10:58:00,servi de point fr pourquoi venir chez castoram...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 172241628202...",1,523
26b53013a4e78c907b56e884c01930963d66390e6d431463c87ea361cea08771,2024-07-31 11:00:00,triathlon et la dix neuvième médaille pour la ...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""changement_climatique_consequences"", ""biodiv...","[{""keyword"": ""alimentation"", ""timestamp"": 1722...",2,566


In [4]:
# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(20)


# Decorator that ensures `acompletion` uses the semaphore
def with_semaphore(acquire_semaphore):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            async with acquire_semaphore:
                return await func(*args, **kwargs)

        return wrapper

    return decorator


acompletion = with_semaphore(semaphore)(acompletion)


class MediatreePrediction(BaseModel):
    label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


async def predict_cards_label_claude(text: str) -> MediatreePrediction:
    system_prompt = """Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Parmis certains extraits, nous avons inséré un passage climatosceptique qui doit être factchecker. Ta tâche est de trouver ce passage climatosceptique (si il existe). Isole le passage à factchecker, puis fournis le label de la catégorie associé à ce passage. Ne t'intéresse qu'aux passages qui sont clairement climatosceptiques et ignore les thématiques sociales et/ou économiques. 

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables.
</catégories prédéfinies>

<exemples>

<exemple_0>
<extrait>
les scientifiques observent une hausse des températures mondiales, entraînant des changements climatiques visibles ils soulignent la nécessité d'agir pour en limiter les impacts [...]
</extrait>
<passage climatosceptique>
AUCUN
</passage climatosceptique>
<catégorie>0_accepted</catégorie>
</exemple_0>

<exemple_1>
<extrait>
[...]
</extrait>
<passage climatosceptique>
la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques
</passage climatosceptique>
<catégorie>1_its_not_happening</catégorie>
</exemple_1>

<exemple_2>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz
</passage climatosceptique>
<catégorie>2_its_not_us</catégorie>
</exemple_2>

<exemple_3>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques
</passage climatosceptique>
<catégorie>3_its_not_bad</catégorie>
</exemple_3>

<exemple_4>
<extrait>
[...]
</extrait>
<passage climatosceptique>
de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement
</passage climatosceptique>
<catégorie>4_solutions_wont_work</catégorie>
</exemple_4>

<exemple_5>
<extrait>
[...]
</extrait>
<passage climatosceptique>
En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée
</passage climatosceptique>
<catégorie>5_science_is_unreliable</catégorie>
</exemple_5>

</exemples>
"""
    # Retry if errors
    while True:
        try:
            response = await acompletion(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {
                        "role": "user",
                        "content": f"<extrait>\n{text.strip()}\n</extrait>",
                    },
                    {
                        "role": "assistant",
                        "content": "<passage climatosceptique>",
                        "prefix": True,
                    },
                ],
                model="anthropic.claude-3-haiku-20240307-v1:0",
                max_tokens=1000,
                temperature=0,
                stop=["</catégorie>"],
            )
            break
        except Exception:
            print("Error, retrying...")

    # Parse the response
    response_content: str = response.choices[0].message.content
    claim = response_content.split("</passage climatosceptique>")[0].strip()
    label_pred = response_content.split("<catégorie>")[1].strip()
    return MediatreePrediction(
        label_pred=label_pred,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        total_tokens=response.usage.total_tokens,
        cost=completion_cost(response),
        claim_pred=claim if label_pred != "0_accepted" else None,
    )

In [4]:
# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(100)


# Decorator that ensures `acompletion` uses the semaphore
def with_semaphore(acquire_semaphore):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            async with acquire_semaphore:
                return await func(*args, **kwargs)

        return wrapper

    return decorator


acompletion = with_semaphore(semaphore)(acompletion)


class MediatreePrediction(BaseModel):
    label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


async def predict_cards_label_openai(text: str) -> MediatreePrediction:
    system_prompt = """Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Parmis certains extraits, nous avons inséré un passage climatosceptique qui doit être factchecker. Ta tâche est de trouver ce passage climatosceptique (si il existe). Isole le passage à factchecker, puis fournis le label de la catégorie associé à ce passage. Ne t'intéresse qu'aux passages qui sont clairement climatosceptiques et ignore les thématiques sociales et/ou économiques. 

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables.
</catégories prédéfinies>

<exemples>

<exemple_0>
<extrait>
les scientifiques observent une hausse des températures mondiales, entraînant des changements climatiques visibles ils soulignent la nécessité d'agir pour en limiter les impacts [...]
</extrait>
<passage climatosceptique>
AUCUN
</passage climatosceptique>
<catégorie>0_accepted</catégorie>
</exemple_0>

<exemple_1>
<extrait>
[...]
</extrait>
<passage climatosceptique>
la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques
</passage climatosceptique>
<catégorie>1_its_not_happening</catégorie>
</exemple_1>

<exemple_2>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz
</passage climatosceptique>
<catégorie>2_its_not_us</catégorie>
</exemple_2>

<exemple_3>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques
</passage climatosceptique>
<catégorie>3_its_not_bad</catégorie>
</exemple_3>

<exemple_4>
<extrait>
[...]
</extrait>
<passage climatosceptique>
de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement
</passage climatosceptique>
<catégorie>4_solutions_wont_work</catégorie>
</exemple_4>

<exemple_5>
<extrait>
[...]
</extrait>
<passage climatosceptique>
En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée
</passage climatosceptique>
<catégorie>5_science_is_unreliable</catégorie>
</exemple_5>

</exemples>
"""
    # Retry if errors
    while True:
        try:
            response = await acompletion(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {
                        "role": "user",
                        "content": f"<extrait>\n{text.strip()}\n</extrait>",
                    },
                    # {
                    #     "role": "assistant",
                    #     "content": "<passage climatosceptique>",
                    #     "prefix": True,
                    # },
                ],
                # model="anthropic.claude-3-haiku-20240307-v1:0",
                model="gpt-4o-mini",
                max_tokens=1000,
                temperature=0,
                stop=["</catégorie>"],
            )
            break
        except Exception:
            print("Error, retrying...")

    # Parse the response
    response_content: str = response.choices[0].message.content
    claim = (
        response_content.split("<passage climatosceptique>")[1]
        .split("</passage climatosceptique>")[0]
        .strip()
    )
    label_pred = response_content.split("<catégorie>")[1].strip()
    return MediatreePrediction(
        label_pred=label_pred,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        total_tokens=response.usage.total_tokens,
        cost=completion_cost(response),
        claim_pred=claim if label_pred != "0_accepted" else None,
    )

In [5]:
# Make predictions
mediatree_predictions: list[MediatreePrediction] = await tqdm.gather(
    *[predict_cards_label_claude(text) for text in df["text"]]
)
mediatree_predictions_df = pd.DataFrame(
    [pred.model_dump(exclude_none=True) for pred in mediatree_predictions],
    index=df.index,
)
df = pd.concat([df, mediatree_predictions_df], axis=1)

100%|██████████| 13760/13760 [33:41<00:00,  6.81it/s] 


In [6]:
df["label_pred"].value_counts()

label_pred
0_accepted                 13298
4_solutions_wont_work        189
3_its_not_bad                 90
5_science_is_unreliable       84
2_its_not_us                  74
1_its_not_happening           25
Name: count, dtype: int64

In [7]:
df["label_pred"].value_counts()

label_pred
0_accepted                 13298
4_solutions_wont_work        189
3_its_not_bad                 90
5_science_is_unreliable       84
2_its_not_us                  74
1_its_not_happening           25
Name: count, dtype: int64

In [10]:
(df["label_pred"] != "0_accepted").sum()

np.int64(462)

In [8]:
(df["label_pred"] != "0_accepted").sum()

np.int64(1428)

In [13]:
# Save the result
df.to_csv(
    "../../data/experiments/label_studio_review/mediatree_predictions_07_2024.csv"
)

In [14]:
def show_llm_usage(df: pd.DataFrame) -> None:
    print("\nLLM USAGE\n=========\n")
    print(
        f"Median token usage:\n- Prompt: {int(df["prompt_tokens"].median())}\n"
        f"- Completion: {int(df["completion_tokens"].median())}\n"
        f"- Total: {int(df["total_tokens"].median())}"
    )
    print(f"\nTotal cost: ${df["cost"].sum():.3f}\n")


show_llm_usage(df)


LLM USAGE

Median token usage:
- Prompt: 1790
- Completion: 27
- Total: 1820

Total cost: $6.658



In [15]:
df["label_pred"].value_counts()

label_pred
0_accepted                 13298
4_solutions_wont_work        189
3_its_not_bad                 90
5_science_is_unreliable       84
2_its_not_us                  74
1_its_not_happening           25
Name: count, dtype: int64

## Manually review the predictions with LabelStudio


In [16]:
df = pd.read_csv(
    "../../data/experiments/label_studio_review/mediatree_predictions_07_2024.csv",
    index_col=0,
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,label_pred,prompt_tokens,completion_tokens,total_tokens,cost,claim_pred
id,,,,,,,,,,,,,,,,
ae3110362b2d90e3b39022926052cb8c57d52be76f692dfccb179236a64fdefd,2024-01-01 06:02:00,sur la boisson maintenant que les fêtes sont a...,europe1,True,Information - Magazine,Bonjour,"[""ressources"", ""changement_climatique_constat""...","[{""keyword"": ""transition \u00e9cologique"", ""ti...",1,524,0_accepted,1786,27,1813,0.000480,NaN
068edefaad4938b6cf7a86bf8e306f6968bbbf3c96e84627b6295900ed9d5615,2024-01-01 06:14:00,première prescription culture de l'année c'est...,europe1,True,Information - Magazine,Bonjour,"[""changement_climatique_constat"", ""biodiversit...","[{""keyword"": ""ademe"", ""timestamp"": 17040861260...",1,497,0_accepted,1757,27,1784,0.000473,NaN
f647d3fad5acbe70ee52b488bd3e8ec8be1018acc4e7756b48bb29b81b877e50,2024-01-01 06:36:00,hamas hier soir c'était donc un réveillon dans...,europe1,True,Information - Magazine,Bonjour,"[""attenuation_climatique_solutions"", ""biodiver...","[{""keyword"": ""sobri\u00e9t\u00e9"", ""timestamp""...",1,496,0_accepted,1763,27,1790,0.000474,NaN
d346e52092308430d6ec6faa668722219ce024db394c112e5e79ba9b8352ff89,2024-01-01 06:54:00,d'accélérer cette part du made in france mais ...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""attenuatio...","[{""keyword"": ""emballage"", ""timestamp"": 1704088...",1,488,0_accepted,1778,27,1805,0.000478,NaN
cb1c730c8eabec84989e98a517db234d22b495d3e0814a9a6286dd61f949d89c,2024-01-01 07:02:00,qui porte bonheur pour moi le quatre août deux...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""feux"", ""timestamp"": 170408896200...",1,528,0_accepted,1800,27,1827,0.000484,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32b48450801f677a3834cb84281b9d91d974f8c8d3ca7e84bed103d7218566de,2024-07-31 10:04:00,dans toute la france le bureau de recherches g...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""changement_climatique_constat"", ""changement_...","[{""keyword"": ""agriculteur"", ""timestamp"": 17224...",4,530,0_accepted,1769,27,1796,0.000476,NaN
2750001e70a1cd4eae8ac6932a495b7ac97feeab1e9d747a53a792b79d4d6a5d,2024-07-31 10:58:00,servi de point fr pourquoi venir chez castoram...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 172241628202...",1,523,0_accepted,1834,27,1861,0.000492,NaN
26b53013a4e78c907b56e884c01930963d66390e6d431463c87ea361cea08771,2024-07-31 11:00:00,triathlon et la dix neuvième médaille pour la ...,sud-radio,True,Information - Magazine,Mettez-vous d'accord,"[""changement_climatique_consequences"", ""biodiv...","[{""keyword"": ""alimentation"", ""timestamp"": 1722...",2,566,0_accepted,1841,27,1868,0.000494,NaN


In [17]:
# Generated code!
def approximate_match(text: str, claim: str, threshold=0.6) -> tuple[int, int] | None:
    # Preprocess text to remove casing and accents
    text = unidecode(text.lower())
    claim = claim.lower()

    # Split text and claim into words
    text_words = text.split()
    claim_words = claim.split()

    # Initialize variables to keep track of best match
    best_match_score = 0
    best_match_range = (None, None)

    # Slide a window of claim length over text to find the best match
    for i in range(len(text_words) - len(claim_words) + 1):
        # Extract a slice of words from the text to compare with claim
        window = text_words[i : i + len(claim_words)]

        # Calculate similarity ratio between the window and claim
        similarity = difflib.SequenceMatcher(None, window, claim_words).ratio()

        # Check if the similarity is above the threshold and better than the previous best
        if similarity > threshold and similarity > best_match_score:
            best_match_score = similarity
            best_match_range = (i, i + len(claim_words) - 1)

    # Convert word indices to character indices
    if best_match_range != (None, None):
        start_word_idx, end_word_idx = best_match_range
        start_char_idx = len(" ".join(text_words[:start_word_idx])) + (
            1 if start_word_idx > 0 else 0
        )
        end_char_idx = len(" ".join(text_words[: end_word_idx + 1]))
        return (start_char_idx, end_char_idx)
    else:
        return None

In [18]:
# Format for LabelStudio
# Only review flagged transcripts
data = []
for id, row in df[df["label_pred"] != "0_accepted"].iterrows():
    # Find the extracted claim in the original text
    text: str = row["text"]
    claim_pred: str = row["claim_pred"]
    try:
        # Try to see if we have a perfect match of claim in text
        start = text.lower().index(claim_pred.lower())
        end = start + len(claim_pred)
    except ValueError:
        # ValueError: substring not found
        matching_result = approximate_match(text, claim_pred, threshold=0.5)
        if matching_result:
            start, end = matching_result
        else:
            # The LLM "extracted" a claim that doesn't exist in the text...
            start, end = None, None

    # Pre-annotate the task with the model's prediction
    # We skip it if we couldn't match the extracted claim with the original text
    if start is not None:
        data.append(
            {
                "data": {
                    "id": id,
                    "start": str(row["start"]),
                    "text": text,
                    "channel_name": row["channel_name"],
                    "channel_program_type": row["channel_program_type"],
                    "channel_program": row["channel_program"],
                    "label_pred": row["label_pred"],
                    "claim_pred": claim_pred,
                },
                "predictions": [
                    {
                        "model_version": "claude-3-haiku",
                        "result": [
                            {
                                "value": {
                                    "start": start,
                                    "end": end,
                                    "text": text[start:end],
                                    "labels": [row["label_pred"]],
                                },
                                "from_name": "label",
                                "to_name": "text",
                                "type": "labels",
                            }
                        ],
                    }
                ],
            }
        )

In [19]:
# Save the LabelStudio tasks for import
with open(
    "../../data/experiments/label_studio_review/mediatree_predictions_07_2024_label_studio_review.json",
    "w",
) as f:
    json.dump(data, f)